# Custom MCP Agent with Inventory Management

This notebook demonstrates how to create an AI agent that can interact with custom Model Context Protocol (MCP) tools for inventory management using Semantic Kernel.

## Import the Needed Packages

We'll import all the necessary libraries for creating our custom MCP-enabled agent.

In [2]:
import os
import asyncio
import json
from dotenv import load_dotenv

# Semantic Kernel imports
from azure.identity import DefaultAzureCredential
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.connectors.mcp import MCPStdioPlugin

from IPython.display import display, HTML

## Setting up Azure OpenAI Connection

Configure the Azure OpenAI service using Azure AD authentication.

In [14]:
# Load environment variables from .env file
load_dotenv()

# Create Azure credential
credential = DefaultAzureCredential()

# Create a token provider function
def get_azure_ad_token():
    """Function to get Azure AD token for OpenAI."""
    token = credential.get_token("https://cognitiveservices.azure.com/.default")
    return token.token

# Create an AI Service using AzureChatCompletion with DefaultAzureCredential
chat_completion_service = AzureChatCompletion(
    deployment_name=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o"),
    endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2025-01-01-preview"),
    ad_token_provider=get_azure_ad_token
)

print("Azure OpenAI service configured successfully!")

Azure OpenAI service configured successfully!


## Configure Custom MCP Plugin

Set up the Model Context Protocol plugin to connect to our custom inventory management server.

In [4]:
# Create MCP Plugin for stdio connection to our custom server
# Note: name must follow pattern ^[0-9A-Za-z_]+$
mcp_plugin = MCPStdioPlugin(
    name="inventory_server",
    command="python",
    args=["server.py"],
    description="Inventory management tools"
)

# Create kernel
kernel = Kernel()

print(f"MCP Plugin configured for custom inventory server")

MCP Plugin configured for custom inventory server


## Creating the Custom MCP Agent

Create an AI agent with access to our custom inventory management tools through MCP.

In [5]:
# Create agent with MCP plugin via kernel
agent = ChatCompletionAgent(
    service=chat_completion_service,
    kernel=kernel,
    name="Inventory-Management-Agent",
    instructions="""
    You are an inventory assistant with access to inventory management tools through MCP.
    
    Here are some general guidelines:
    - Recommend restock if item inventory < 10 and weekly sales > 15
    - Recommend clearance if item inventory > 20 and weekly sales < 5
    
    Use the available MCP tools to get inventory levels, weekly sales data, and product details
    to help answer user questions about inventory management. Always provide specific recommendations
    based on the data you retrieve.
    
    Available tools:
    - get_inventory_levels: Returns current inventory for all products
    - get_weekly_sales: Returns number of units sold last week
    - get_product_details: Returns details for a specific product
    
    When providing inventory analysis, always show the data you retrieved and explain your reasoning.
    """
)

print(f"Created agent: {agent.name}")

Created agent: Inventory-Management-Agent


## Initialize Custom MCP Connection

Connect to our custom MCP server and load available tools.

In [6]:
async def initialize_custom_mcp():
    """Initialize custom MCP plugin connection and load tools."""
    try:
        # Connect to MCP server
        await mcp_plugin.connect()
        print(f"✓ Custom MCP Plugin connected successfully")
        
        # Load tools and prompts
        await mcp_plugin.load_tools()
        print("✓ Custom MCP tools loaded")
        
        # Add the MCP plugin to the kernel
        kernel.add_plugin(mcp_plugin)
        print(f"✓ Custom MCP plugin added to kernel")
        
        # Show available functions
        print("\nAvailable custom MCP tools:")
        for plugin_name, plugin in kernel.plugins.items():
            for function_name, function in plugin.functions.items():
                print(f"  - {function_name}: {function.description}")
        
        return True
        
    except Exception as e:
        print(f"Custom MCP Plugin setup error: {e}")
        import traceback
        traceback.print_exc()
        return False

# Initialize the custom MCP connection
mcp_initialized = await initialize_custom_mcp()
print(f"\nCustom MCP initialization {'successful' if mcp_initialized else 'failed'}")

✓ Custom MCP Plugin connected successfully
✓ Custom MCP tools loaded
✓ Custom MCP plugin added to kernel

Available custom MCP tools:
  - get_inventory_levels: Returns current inventory for all products.
  - get_product_details: Returns details for a specific product.
  - get_weekly_sales: Returns number of units sold last week.

Custom MCP initialization successful


## Agent Interaction Function

Create a function to handle user interactions with the agent, displaying both the conversation and function calls.

In [7]:
async def chat_with_agent(user_input, thread=None):
    """Chat with the agent and display results with function call details."""
    
    html_output = (
        f"<div style='margin-bottom:10px'>"
        f"<div style='font-weight:bold'>User:</div>"
        f"<div style='margin-left:20px'>{user_input}</div></div>"
    )

    agent_name = None
    full_response: list[str] = []
    function_calls: list[str] = []

    # Buffer to reconstruct streaming function call
    current_function_name = None
    argument_buffer = ""

    try:
        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args) if formatted_args else {}
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"🔧 Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"📊 Function Result:\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        # Display function calls if any were made
        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>MCP Tool Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))
        return thread
        
    except Exception as e:
        print(f"Error during agent execution: {e}")
        import traceback
        traceback.print_exc()
        return thread

print("Agent interaction function ready!")

Agent interaction function ready!


## Test the Inventory Agent

Now let's test our custom MCP-enabled agent with some inventory management questions.

In [8]:
# Test questions for the inventory agent
test_questions = [
    #"What are the current inventory levels for all products?",
    #"Which products should I restock based on current inventory and sales?",
    "Show me the weekly sales data and identify slow-moving items"
]

# Run the conversation
thread = None
for question in test_questions:
    thread = await chat_with_agent(question, thread)
    print("\n" + "="*80 + "\n")  # Separator between questions

In [9]:
# Detailed analysis question
detailed_question = "Can you give me detailed information about Moisturizer including its product details, current inventory, and sales performance? Should I take any action?"
thread = None
thread = await chat_with_agent(detailed_question, thread)

## Interactive Chat

Use this cell to ask your own questions to the inventory agent. The agent will use the custom MCP tools to provide accurate inventory management insights.

In [10]:
# Interactive chat - modify the question below
user_question = "What products are performing well and which ones need attention?"

# Use the existing thread to maintain conversation context
thread = await chat_with_agent(user_question, thread)

## Custom Inventory Query

Create your own custom query to test the agent's capabilities.

In [11]:
# Custom query - replace with your own question
custom_question = "Give me a complete inventory report with recommendations for each product category"

thread = await chat_with_agent(custom_question, thread)

## View Available MCP Tools

Let's examine what tools are available from our custom MCP server.

In [12]:
# Display available tools and their descriptions
print("Available Custom MCP Tools:")
print("=" * 50)

for plugin_name, plugin in kernel.plugins.items():
    print(f"\nPlugin: {plugin_name}")
    for function_name, function in plugin.functions.items():
        print(f"  🔧 {function_name}")
        print(f"     Description: {function.description}")
        print(f"     Parameters: {[param.name for param in function.parameters]}")

Available Custom MCP Tools:

Plugin: inventory_server
  🔧 get_inventory_levels
     Description: Returns current inventory for all products.
     Parameters: []
  🔧 get_product_details
     Description: Returns details for a specific product.
     Parameters: ['product_name']
  🔧 get_weekly_sales
     Description: Returns number of units sold last week.
     Parameters: []


## Cleanup

Clean up the MCP connection when done.

In [13]:
# Clean up MCP connection
try:
    await mcp_plugin.close()
    print("✓ Custom MCP Plugin closed successfully")
except Exception as e:
    print(f"Error closing custom MCP plugin: {e}")

Error closing custom MCP plugin: Attempted to exit cancel scope in a different task than it was entered in
